<a href="https://colab.research.google.com/github/CanopySimulations/canopy-python-examples/blob/master/running_studies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upgrade Runtime
This cell ensures the runtime supports `asyncio` async/await, and is needed on Google Colab. If the runtime is upgraded, you will be prompted to restart it, which you should do before continuing execution.

In [0]:
!pip install "ipython>=7"

# Set Up Environment

### Import required libraries

In [0]:
!pip install -q 'canopy==6.2'

In [0]:
import canopy
import logging

logging.basicConfig(level=logging.INFO)

### Authenticate

In [0]:
authentication_data = canopy.prompt_for_authentication()

def create_session():
    return canopy.Session(authentication_data)

# Set Up Example

Create the input configs for our study:

In [0]:
async with create_session() as session:
    # Load a default car and save it as a user config.
    default_car = await canopy.load_default_config(
        session, 'car', 'Canopy F1 Car 2019')
    
    user_car_id = await canopy.create_config(
        session,
        'car',
        'Running Studies Example Car',
        default_car.raw_data)
    
    # Load a default weather and save it as a user config.
    default_weather = await canopy.load_default_config(
        session, 'weather', '25 deg, dry')
    
    user_weather_id = await canopy.create_config(
        session,
        'weather',
        'Running Studies Example Weather',
        default_weather.raw_data)

    # Load the user weather config.    
    user_weather = await canopy.load_config(session, user_weather_id)
    
    # Load a default exploration and reduce its size for this example.
    default_exploration = await canopy.load_default_config(
        session, 'exploration', 'Automated Test Monte Carlo')
    default_exploration.data.design.numberOfPoints = 3

> **Tip: `data` vs `raw_data`**
>
> We could use either the `default_car.raw_data` property or the `default_car.data` property when creating the user config above. 
> 
> On first access, the `data` property will convert the nested dictionaries (`car['a']['b']`) into a structure which we can manipulate more easily (`car.a.b`). 
> 
> While convenient, this conversion does have some overhead, particularly on large configs such as tracks. As we are not manipulating the data it is good practice to skip the conversion by using the `raw_data` property instead. 
> 
> The `raw_data` property will return either:
 - The same structure as `data` if the conversion has already been done.
 - The original raw nested dictionaries if the conversion hasn't already been done.




# Example: Running Studies

The `create_study` function takes a list of configs to use as inputs to the study. Each config can be either:
 - A string config ID.
 - A `ConfigResult` class, as returned by functions such as `load_config`.
 - A `LocalConfig` class, as returned by functions such as `load_default_config`.

The code below demonstrates passing one config of each type:

In [6]:
async with create_session() as session:
    study_id = await canopy.create_study(
        session,
        'apexSim',
        'Running Studies Example Study',
        [
            user_car_id, # String
            user_weather, # ConfigResult class
            default_exploration, # LocalConfig class
        ])
    
study_id

INFO:canopy.create_study:Loaded input config car


'ba986c5ce1c945c3a1a97ed8a22c6efc'

> **Tip: Study Types and Sim Types**
>
> A study contains one or more jobs, and each job contains one or more simulations.
For example a Dynamic Lap study will contain a Dynamic Lap, a Straight Sim and Apex Sim simulation in each job.
> 
> When dealing with the Canopy API study types are `camelCase` and sim types are `PascalCase`.
>
> Therefore a `dynamicLap` study will contain `DynamicLap`, `StraightSim` and `ApexSim` simulations.
>
> Above we are running an `apexSim` study which will contain `ApexSim` and `StraightSim` simulations.
>
> The Python library will tend to ignore case in the helper functions we've written and interpret the string based on the context, however when dealing with the generated Swagger client code directly you will need to use the correct case so it is good practice to always try and do this.


In [9]:
async with create_session() as session:
    # If we want to access the study results then we can 
    # wait for the study to complete.
    wait_result = await canopy.wait_for_study(
        session,
        study_id,
        timeout_seconds=300)
    
logging.info('Sim Version: {}'.format(wait_result.document.sim_version))    
logging.info('Succeded Job Count: {}/{}'.format(
    wait_result.data.succeeded_job_count, 
    wait_result.data.job_count))

INFO:root:Sim Version: 1.3037
INFO:root:Succeded Job Count: 4/4
